<a href="https://colab.research.google.com/github/ammfat/pyspark-zero-to-hero/blob/colab/04_string_and_dates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
# Spark Session
from pyspark.sql import SparkSession

spark = (
    SparkSession
    .builder
    .appName("Working with Strings & Dates")
    .master("local[*]")
    .getOrCreate()
)

spark

In [24]:
# Emp Data & Schema

emp_data = [
    ["001","101","John Doe","30","Male","50000","2015-01-01"],
    ["002","101","Jane Smith","25","Female","45000","2016-02-15"],
    ["003","102","Bob Brown","35","Male","55000","2014-05-01"],
    ["004","102","Alice Lee","28","Female","48000","2017-09-30"],
    ["005","103","Jack Chan","40","Male","60000","2013-04-01"],
    ["006","103","Jill Wong","32","Female","52000","2018-07-01"],
    ["007","101","James Johnson","42","Male","70000","2012-03-15"],
    ["008","102","Kate Kim","29","Female","51000","2019-10-01"],
    ["009","103","Tom Tan","33","Male","58000","2016-06-01"],
    ["010","104","Lisa Lee","27","Female","47000","2018-08-01"],
    ["011","104","David Park","38","Male","65000","2015-11-01"],
    ["012","105","Susan Chen","31","Female","54000","2017-02-15"],
    ["013","106","Brian Kim","45","Male","75000","2011-07-01"],
    ["014","107","Emily Lee","26","Female","46000","2019-01-01"],
    ["015","106","Michael Lee","37","Male","63000","2014-09-30"],
    ["016","107","Kelly Zhang","30","Female","49000","2018-04-01"],
    ["017","105","George Wang","34","Male","57000","2016-03-15"],
    ["018","104","Nancy Liu","29","","50000","2017-06-01"],
    ["019","103","Steven Chen","36","Male","62000","2015-08-01"],
    ["020","102","Grace Kim","32","Female","53000","2018-11-01"]
]

emp_schema = "employee_id string, department_id string, name string, age string, gender string, salary string, hire_date string"

In [25]:
# Create emp DataFrame

emp = spark.createDataFrame(data=emp_data, schema=emp_schema)

In [26]:
# Show emp dataframe (ACTION)

emp.show()

+-----------+-------------+-------------+---+------+------+----------+
|employee_id|department_id|         name|age|gender|salary| hire_date|
+-----------+-------------+-------------+---+------+------+----------+
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|
|        006|          103|    Jill Wong| 32|Female| 52000|2018-07-01|
|        007|          101|James Johnson| 42|  Male| 70000|2012-03-15|
|        008|          102|     Kate Kim| 29|Female| 51000|2019-10-01|
|        009|          103|      Tom Tan| 33|  Male| 58000|2016-06-01|
|        010|          104|     Lisa Lee| 27|Female| 47000|2018-08-01|
|        011|          104|   David Park| 38|  Male| 65000|2015-11-01|
|     

In [27]:
# Print Schema

emp.printSchema()

root
 |-- employee_id: string (nullable = true)
 |-- department_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: string (nullable = true)
 |-- hire_date: string (nullable = true)



In [28]:
# Case When
# select employee_id, name, age, salary, gender,
# case when gender = 'Male' then 'M' when gender = 'Female' then 'F' else null end as new_gender, hire_date from emp
from pyspark.sql.functions import when, col, expr

emp_gender_fixed = emp.withColumn("new_gender", when(col("gender") == 'Male', 'M')
                                 .when(col("gender") == 'Female', 'F')
                                 .otherwise(None)
                                 )

emp_gender_fixed_1 = emp.withColumn("new_gender", expr("case when gender = 'Male' then 'M' when gender = 'Female' then 'F' else null end"))

In [29]:
emp_gender_fixed_1.show()

+-----------+-------------+-------------+---+------+------+----------+----------+
|employee_id|department_id|         name|age|gender|salary| hire_date|new_gender|
+-----------+-------------+-------------+---+------+------+----------+----------+
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|         M|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|         F|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|         M|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|         F|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|         M|
|        006|          103|    Jill Wong| 32|Female| 52000|2018-07-01|         F|
|        007|          101|James Johnson| 42|  Male| 70000|2012-03-15|         M|
|        008|          102|     Kate Kim| 29|Female| 51000|2019-10-01|         F|
|        009|          103|      Tom Tan| 33|  Male| 58000|2016-06-01|         M|
|        010|   

In [30]:
# Replace in Strings
# select employee_id, name, replace(name, 'J', 'Z') as new_name, age, salary, gender, new_gender, hire_date from emp_gender_fixed
from pyspark.sql.functions import regexp_replace

emp_name_fixed = emp_gender_fixed.withColumn("new_name", regexp_replace(col("name"), "J", "Z"))


In [31]:
emp_name_fixed.show()

+-----------+-------------+-------------+---+------+------+----------+----------+-------------+
|employee_id|department_id|         name|age|gender|salary| hire_date|new_gender|     new_name|
+-----------+-------------+-------------+---+------+------+----------+----------+-------------+
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|         M|     Zohn Doe|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|         F|   Zane Smith|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|         M|    Bob Brown|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|         F|    Alice Lee|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|         M|    Zack Chan|
|        006|          103|    Jill Wong| 32|Female| 52000|2018-07-01|         F|    Zill Wong|
|        007|          101|James Johnson| 42|  Male| 70000|2012-03-15|         M|Zames Zohnson|
|        008|          102|     Kate Kim

In [32]:
# Convert Date
# select *,  to_date(hire_date, 'YYYY-MM-DD') as hire_date from emp_name_fixed
from pyspark.sql.functions import to_date

emp_date_fix = emp_name_fixed.withColumn("hire_date", to_date(col("hire_date"), 'yyyy-MM-dd'))


In [33]:
emp_date_fix.printSchema()

root
 |-- employee_id: string (nullable = true)
 |-- department_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: string (nullable = true)
 |-- hire_date: date (nullable = true)
 |-- new_gender: string (nullable = true)
 |-- new_name: string (nullable = true)



In [34]:
# Add Date Columns
# Add current_date, current_timestamp, extract year from hire_date
from pyspark.sql.functions import current_date, current_timestamp

emp_dated = emp_date_fix.withColumn("date_now", current_date()).withColumn("timestamp_now", current_timestamp())


In [35]:
emp_dated.show(truncate=False)

+-----------+-------------+-------------+---+------+------+----------+----------+-------------+----------+--------------------------+
|employee_id|department_id|name         |age|gender|salary|hire_date |new_gender|new_name     |date_now  |timestamp_now             |
+-----------+-------------+-------------+---+------+------+----------+----------+-------------+----------+--------------------------+
|001        |101          |John Doe     |30 |Male  |50000 |2015-01-01|M         |Zohn Doe     |2025-08-14|2025-08-14 19:39:36.683717|
|002        |101          |Jane Smith   |25 |Female|45000 |2016-02-15|F         |Zane Smith   |2025-08-14|2025-08-14 19:39:36.683717|
|003        |102          |Bob Brown    |35 |Male  |55000 |2014-05-01|M         |Bob Brown    |2025-08-14|2025-08-14 19:39:36.683717|
|004        |102          |Alice Lee    |28 |Female|48000 |2017-09-30|F         |Alice Lee    |2025-08-14|2025-08-14 19:39:36.683717|
|005        |103          |Jack Chan    |40 |Male  |60000 |201

In [36]:
# Drop Null gender records
emp_1 = emp_dated.na.drop()


In [37]:
emp_1.show()

+-----------+-------------+-------------+---+------+------+----------+----------+-------------+----------+--------------------+
|employee_id|department_id|         name|age|gender|salary| hire_date|new_gender|     new_name|  date_now|       timestamp_now|
+-----------+-------------+-------------+---+------+------+----------+----------+-------------+----------+--------------------+
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|         M|     Zohn Doe|2025-08-14|2025-08-14 19:39:...|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|         F|   Zane Smith|2025-08-14|2025-08-14 19:39:...|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|         M|    Bob Brown|2025-08-14|2025-08-14 19:39:...|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|         F|    Alice Lee|2025-08-14|2025-08-14 19:39:...|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|         M|    Zack Chan|2025-08-1

In [38]:
# Fix Null values
# select *, nvl('new_gender', 'O') as new_gender from emp_dated
from pyspark.sql.functions import coalesce, lit

emp_null_df = emp_dated.withColumn("new_gender", coalesce(col("new_gender"), lit("O")))


In [39]:
emp_null_df.show()

+-----------+-------------+-------------+---+------+------+----------+----------+-------------+----------+--------------------+
|employee_id|department_id|         name|age|gender|salary| hire_date|new_gender|     new_name|  date_now|       timestamp_now|
+-----------+-------------+-------------+---+------+------+----------+----------+-------------+----------+--------------------+
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|         M|     Zohn Doe|2025-08-14|2025-08-14 19:39:...|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|         F|   Zane Smith|2025-08-14|2025-08-14 19:39:...|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|         M|    Bob Brown|2025-08-14|2025-08-14 19:39:...|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|         F|    Alice Lee|2025-08-14|2025-08-14 19:39:...|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|         M|    Zack Chan|2025-08-1

In [40]:
# Drop old columns and Fix new column names
emp_final = emp_null_df.drop("name", "gender").withColumnRenamed("new_name", "name").withColumnRenamed("new_gender", "gender")


In [41]:
emp_final.show()

+-----------+-------------+---+------+----------+------+-------------+----------+--------------------+
|employee_id|department_id|age|salary| hire_date|gender|         name|  date_now|       timestamp_now|
+-----------+-------------+---+------+----------+------+-------------+----------+--------------------+
|        001|          101| 30| 50000|2015-01-01|     M|     Zohn Doe|2025-08-14|2025-08-14 19:39:...|
|        002|          101| 25| 45000|2016-02-15|     F|   Zane Smith|2025-08-14|2025-08-14 19:39:...|
|        003|          102| 35| 55000|2014-05-01|     M|    Bob Brown|2025-08-14|2025-08-14 19:39:...|
|        004|          102| 28| 48000|2017-09-30|     F|    Alice Lee|2025-08-14|2025-08-14 19:39:...|
|        005|          103| 40| 60000|2013-04-01|     M|    Zack Chan|2025-08-14|2025-08-14 19:39:...|
|        006|          103| 32| 52000|2018-07-01|     F|    Zill Wong|2025-08-14|2025-08-14 19:39:...|
|        007|          101| 42| 70000|2012-03-15|     M|Zames Zohnson|202

In [42]:
# Write data as CSV
emp_final.write.format("csv").save("data/output/4/emp.csv")


AnalysisException: [PATH_ALREADY_EXISTS] Path file:/content/data/output/4/emp.csv already exists. Set mode as "overwrite" to overwrite the existing path.

In [ ]:
# Bonus TIP
# Convert date into String and extract date information
from pyspark.sql.functions import date_format

emp_fixed = emp_final.withColumn("date_year", date_format(col("timestamp_now"), "z"))



In [ ]:
emp_fixed.show()